In [1]:
from gridworld import *
from tqdm import trange
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#R-Learning

gw = gridworld((5, 5), (0, 4), (4, 4), [(o, 4) for o in range(1, 4)], 0.9, 0, 1, 0)
y = 0.95

In [2]:
#Empirically estimate risk probability
num_sims = 10000
def estimate_rr(pi):
    risky = 0
    rewards = []
    discounted = []
    for i in trange(num_sims):
        s = gw.reset()
        reward = 0
        d = False
        j = 0
        while(j < 1000):
            a = pi[s[0],s[1]]
            s1,r,d = gw.step(a)
            reward += r
            if d == True:
                if s1 in gw.error_states:
                    risky += 1
                    discounted_reward = 0
                else:
                    discounted_reward = pow(y, j) * r
                break
            s = s1
            j += 1
        rewards.append(reward)
        discounted.append(discounted_reward)
    return sum(discounted)/float(num_sims), sum(rewards)/float(num_sims), float(risky)/num_sims    

In [3]:
trials = 10
Q_list = []
reward_trial_list = []

np.random.seed(1)

# Set learning parameters
lr = 0.9
num_episodes = 100000
factor = 0.02

for k in trange(trials):
    #Initialize table with all zeros
    Q = np.zeros((gw.M, gw.N, 4))
    Q_ = np.zeros((gw.M, gw.N, 4))
    mainQ = np.zeros((gw.M, gw.N, 4))
    jList = []
    rList = []
    for i in range(num_episodes):
        if i%10000 == 1:
            print('Episode', i)
        #Reset environment and get first new observation
        s = gw.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Table learning algorithm
        while j < 100:
            j+=1
            #Choose an action by greedily (with noise) picking from Q table
            a = np.argmax(Q[s[0],s[1],:] + np.random.randn(1,4)*(1./(i+1)))
#             if np.random.rand() < epsilon:
#                 # take random action
#                 a = np.random.choice([0,1,2,3])
#             else:
#                 # take action according to the q function table
#                 a = np.argmax(Q[s[0],s[1],:])
            #Get new state and reward from environment
            s1,r,d = gw.step(a)

            r_ = 0
            if s1 in gw.error_states:
                r_ = 1

            #Update Q-Table with new knowledge
            Q[s[0], s[1],a] = Q[s[0], s[1],a] + lr*(r + y*np.max(Q[s1[0], s1[1],:]) - Q[s[0], s[1],a])
            Q_[s[0], s[1],a] = Q_[s[0], s[1],a] + lr*(r_ + y*np.max(Q_[s1[0], s1[1],:]) - Q_[s[0], s[1],a])
            mainQ = factor * Q - Q_  

            rAll += r
            s = s1
            if d == True:
                break
        jList.append(j)
        rList.append(rAll)
        
    Q_list.append(mainQ)

#     v = np.mean(mainQ, axis = 2)
#     gw.print_v(v)


    pi = np.argmax(mainQ, axis = 2)
    gw.print_policy(pi)
    reward_trial_list.append(sum(rList)/float(num_episodes))
    #print "Score over time: " +  str(sum(rList)/float(num_episodes))
    
#     window = int(num_episodes/10)

#     plt.figure(figsize=[9,16])
#     plt.subplot(411)
#     plt.plot(pd.Series(jList).rolling(window).mean())
#     plt.title('Step Moving Average ({}-episode window)'.format(window))
#     plt.ylabel('Moves')
#     plt.xlabel('Episode')

#     plt.subplot(412)
#     plt.plot(pd.Series(rList).rolling(window).mean())
#     plt.title('Reward Moving Average ({}-episode window)'.format(window))
#     plt.ylabel('Reward')
#     plt.xlabel('Episode')

#     plt.tight_layout(pad=2)
#     plt.show()
    

gw.print_gw()
meanQ = np.mean(np.array(Q_list), axis = 0)
meanv = np.mean(meanQ, axis = 2)
gw.print_v(meanv)
meanpi = np.argmax(meanQ, axis = 2)
gw.print_policy(meanpi)
E_dis_reward, E_reward, E_risk = estimate_rr(meanpi)
print "Mean reward (empirical)", E_dis_reward
print "Successful Episodes ", E_reward
print "Risk probab ", E_risk

  0%|          | 0/1 [00:00<?, ?it/s]

('Episode', 1)
('Episode', 10001)
('Episode', 20001)
('Episode', 30001)
('Episode', 40001)
('Episode', 50001)
('Episode', 60001)
('Episode', 70001)
('Episode', 80001)
('Episode', 90001)


,0,1,2,3,4
0,⬇,⬅,⬅,⬅,⬅
1,⬇,⬅,⬅,⬅,E
2,⬇,⬅,⬇,⬇,E
3,➡,⬇,⬇,⬇,E
4,➡,➡,⬆,➡,G


100%|██████████| 1/1 [00:21<00:00, 21.85s/it]


,0,1,2,3,4
0,-,-,-,-,S
1,-,-,-,-,E
2,-,-,-,-,E
3,-,-,-,-,E
4,-,-,-,-,G


,0,1,2,3,4
0,-0.74,-0.79,-8.54e-01,-9.14e-01,-0.95
1,-0.73,-0.78,-8.78e-01,-9.28e-01,E
2,-0.69,-0.76,-8.37e-01,-7.08e-01,E
3,-0.64,-0.43,-2.47e-01,4.51e-03,E
4,-0.58,-0.41,-1.26e-03,5.00e-03,G


,0,1,2,3,4
0,⬇,⬅,⬅,⬅,⬅
1,⬇,⬅,⬅,⬅,E
2,⬇,⬅,⬇,⬇,E
3,➡,⬇,⬇,⬇,E
4,➡,➡,⬆,➡,G


100%|██████████| 10000/10000 [00:02<00:00, 4399.87it/s]

Mean reward (empirical) 0.184800172978
Successful Episodes  0.942
Risk probab  0.058
